In [1]:
import nltk
import math
import operator
import functools
import numpy as np
import pandas as pd
from common import utils
from common import prettyprint as pp
from nltk.corpus import wordnet as wn
from scipy.stats.stats import pearsonr, spearmanr

La seguente funzione calcola il percorso di massima lunghezza su WordNet, è piuttosto lento, quindi lo calcolo qui, una volta sola

In [2]:
def depthMax():
    return max(max(len(hyp_path) for hyp_path in ss.hypernym_paths()) for ss in wn.all_synsets())

depth_max = depthMax()

In [3]:
debug = True
def debug_trace(phrase):
    if debug:
        print(phrase)

Le seguenti sono due funzioni alternative per calcolare il lowest common subsumer. La prima (`lcs_orig`) è quella che ho creato interamente da me. La seconda (`lcs_adapted`), è una specializzazione della prima, in cui ho aggiunto alcuni step che (sembra) fare la controparte di WordNet.

In [4]:
# Lowest Common Subsumer
def lcs_orig(c1, c2):
    hypernyms = lambda x: x.hypernyms()
    for i in range(0, max([c1.max_depth(), c2.max_depth()])):
    
        c1_set = set(c1.closure(hypernyms, depth=i))
        c2_set = set(c2.closure(hypernyms, depth=i))
    
        lcs = c1_set.intersection(c2_set)
    
        if (lcs != set()):
            return list(lcs)
    return None

# Lowest Common Subsumer
def lcs_adapted(c1, c2):
    hypernyms = lambda x: x.hypernyms()
    # IDK, nltk does this too
    if c1 == c2:
        return [c1]
    elif c2 in c1.hypernyms():
        return [c2]
    elif c1 in c2.hypernyms():
        return [c1]
    for i in range(0, max([c1.max_depth(), c2.max_depth()])+1):
    
        c1_set = set(c1.closure(hypernyms, depth=i))
        c2_set = set(c2.closure(hypernyms, depth=i))
    
        lcs = c1_set.intersection(c2_set)
    
        if (lcs != set()):
            return list(lcs)
    return None

Seguono le implementazioni delle funzioni richieste.
### Wu-Palmer:
$$\text{cs}(s_1, s_2) = \frac{2 \cdot \text{depth}(LCS)}{\text{depth}(s_1) + \text{depth}(s_2)}$$

In [5]:
def wup_similarity(c1, c2, lcsfun=lcs_adapted):
    c_lcs = lcsfun(c1, c2)
    #print(c_lcs)
    if c_lcs != None:
        return 2*c_lcs[0].max_depth()/(c1.max_depth() + c2.max_depth())
    else:
        return None

### Shortest Path
$$\text{sim}_{\text{path}}(s_1, s_2) = 2 \cdot \text{depthMax} - \text{shortest_path_len}(s_1, s_2)$$


In [6]:
def pth_similarity(c1, c2):
    shortest_len = c1.shortest_path_distance(c2)
    if shortest_len != None:
        return (2*depth_max - shortest_len)
    else:
        return None

### Leacock-Chodorow
$$\text{sim}_{\text{LC}}(s_1, s_2) = -\log \frac{\text{shortest_path_len}(s_1, s_2)}{2 \cdot \text{depthMax}}$$

In [7]:
def lch_similarity(c1, c2):
    shortest_len = c1.shortest_path_distance(c2)
    if shortest_len != None:
        return -math.log((1+shortest_len)/(1+(2*depth_max)))
    else:
        return None

In [8]:
wordsims = pd.read_csv('./data/WordSim353.csv')

La seguente funzione calcola la similarità tra due *termini*, massimizzando il valore di una certa funzione. Anzitutto prende i synsets per entrambe. Calcola il risultato, e se fosse maggiore dell'attuale massimo (inizializzato a 0 per essere rimpiazzato) lo userebbe come nuovo massimo. Quindi, dopo aver iterato su tutte le combinazioni di tutti i sensi, restituisce il risultato con score massimo.

In [9]:
def sim(word1, word2, similarity=wup_similarity):
    ss1 = wn.synsets(word1)
    ss2 = wn.synsets(word2)
    results = list()
    cur_max = (word1,word2,0)
    for i in utils.gen_combo(ss1, ss2):
        result = similarity(i[0], i[1])
        if result != None and result > cur_max[2]:
            cur_max = (i[0], i[1], result)
    return [cur_max]

Nei prossimi blocchi, raccolgo tutti i risultati, calcolati di volta in volta con funzione di similarità differente, e creo un DataFrame (pandas) contenente i risultati calcolati.

In [10]:
wup_series = []
path_series = []
lch_series = []

for i in range(len(wordsims)):
    row = wordsims.iloc[i]

    # find all the senses, then sim = max of them
    wup_series.append(sim(row.Word1, row.Word2, similarity=wup_similarity))
    path_series.append(sim(row.Word1, row.Word2, similarity=pth_similarity))
    lch_series.append(sim(row.Word1, row.Word2, similarity=lch_similarity))



In [11]:
newdata = pd.DataFrame(data={
    'Word1': wordsims.Word1,
    'Word2': wordsims.Word2,
    'Target': wordsims.HumanMean,
    'WUP': list(zip(*[i[0] for i in wup_series]))[2],
    'PTH': list(zip(*[i[0] for i in path_series]))[2],
    'LCH': list(zip(*[i[0] for i in lch_series]))[2],
})

In [12]:
newdata.to_csv('./results/term_similarity.csv')

In [13]:
newdata

,Word1,Word2,Target,WUP,PTH,LCH
0,love,sex,6.77,0.909091,39,3.020425
1,tiger,cat,7.35,0.962963,39,3.020425
2,tiger,tiger,10.00,1.000000,40,3.713572
3,book,paper,7.46,0.857143,38,2.614960
4,computer,keyboard,7.62,0.800000,37,2.327278
...,...,...,...,...,...,...
348,shower,flood,6.03,0.533333,36,2.104134
349,weather,forecast,8.34,0.000000,27,1.074515
350,disaster,area,6.25,0.428571,32,1.516347
351,governor,office,6.34,0.470588,31,1.410987


Nei prossimi blocchi calcolo la correlazione tra i valori target e i valori calcolati, sfruttando le funzioni di scipy.

### Pearson Correlation

In [14]:
print(f"{pp.success(pp.bold('Wu-Palmer'))}: {pearsonr(newdata.WUP, newdata.Target)[0]}")
print(f"{pp.success(pp.bold('Path Similarity'))}: {pearsonr(newdata.PTH, newdata.Target)[0]}")
print(f"{pp.success(pp.bold('Leacock'))}: {pearsonr(newdata.LCH, newdata.Target)[0]}")

Wu-Palmer: 0.2858796939892433
Path Similarity: 0.16653216769600956
Leacock: 0.3143602416139906


### Spearman Correlation

In [15]:
print(f"{pp.success(pp.bold('Wu-Palmer'))}: {spearmanr(newdata.WUP, newdata.Target)[0]}")
print(f"{pp.success(pp.bold('Path Similarity'))}: {spearmanr(newdata.PTH, newdata.Target)[0]}")
print(f"{pp.success(pp.bold('Leacock'))}: {spearmanr(newdata.LCH, newdata.Target)[0]}")

Wu-Palmer: 0.32106636742234207
Path Similarity: 0.2895253335747299
Leacock: 0.2895253335747299


La correlazione è positiva in entrambi i casi: Quando il valore delle funzioni descritte sopra sale, così fa anche il target.

In [16]:
import random
import nltk.wsd
from nltk.corpus import semcor as sc

# Part 2: Word Sense Disambiguation


## Lesk Algorithm Implementation

La prossima funzione calcola la sovrapposizione lessicale tra signature e context. Ho sperimentato la rilevanza del togliere dai rispettivi insiemi di parole (contesto e signature) il termine a cui si riferisce la signature, per controllare se avesse un impatto sulle valutazioni finali o meno (non ce l'ha)

In [17]:
def computeOverlap(signature, context, word=None):
    if len(signature) == 0 or len(context) == 0:
        return 0
    try:
        signature_wordset = set(list(zip(*signature))[0])
        context_wordset = set(list(zip(*context))[0])
        debug_trace(f"{pp.info('INFO')}: signature: {signature_wordset}, context: {context_wordset}")
        totalwords = len(context_wordset)
    except IndexError:
        print(len(context), len(signature))
        print(context)
        print(signature)
    if word != None:
        # remove the target word from both wordsets
        try:
            signature_wordset.remove(word)
            context_wordset.remove(word)
        except KeyError:
            pass
    # +1 to avoid zerodivision error
    return (len(list(signature_wordset.intersection(context_wordset))) +1)/(totalwords+1)

La prossima funzione si occupa di creare il contesto dato un synset, sfruttando esempi e definizioni, sia del dato synset, che dei suoi iperonimi diretti e iponimi diretti, il parametro do_stemming specifica se fare lo stemming dei risultati o meno.

In [18]:
def get_big_disambiguation_ctx_for_sysnet(synset, do_stemming=True):
    # Given a synset, returns a Bag-of-Words model for its definition and examples
    # as well as those of the direct hypernyms and direct hyponyms
    context = synset.definition()
    for example in synset.examples():
        context += example

    for hypernym in synset.hypernyms():
        context += hypernym.definition()
        for hyp_example in hypernym.examples():
            context += hyp_example
    
    for hyponym in synset.hyponyms():
        context += hyponym.definition()
        for hypo_example in hyponym.examples():
            context += hypo_example
    if do_stemming:
        return utils.word_frequencies(utils.preprocess_phrase(context))
    else:
        return utils.word_frequencies(utils.preprocess_phrase_nostem(context))

I prossimi blocchi contengono l'algoritmo di Lesk e alcune specializzazioni che ho testato, per controllare l'impatto sui risultati. L'implementazione del prossimo blocco è quella standard, senza alcun tipo di modifiche, una banale traduzione dallo pseudocodice.

In [19]:
def lesk(word, sentence, overlapf=computeOverlap):
    try:
        best_sense = wn.synsets(word)[0]
    except IndexError:
        best_sense = None
    max_overlap = 0
    context = utils.word_frequencies(utils.preprocess_phrase(sentence))
    for sense in wn.synsets(word):
        debug_trace(f"{pp.info('INFO')}: Considering new sense: {sense}({pp.bold(pp.info(sense.definition()))})")
        signature = utils.word_frequencies(utils.preprocess_phrase(sense.definition()))
        overlap = overlapf(signature, context)
        debug_trace(f"{pp.info('INFO')}: {sense} scored: {pp.success(overlap)}")
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = sense
    return best_sense

Una versione leggermente modificata dell'algoritmo di Lesk, rispetto a quella originale:

1. Aggiunge gli 'esempi' di WordNet come parte della signature (più parole ci sono, più dovrebbe essere facile disambiguare)
2. Rimuove la parola target dalle definizioni (signature e contesto)

In [20]:
def lesk_custom(word, sentence, overlapf=computeOverlap):
    try:
        best_sense = wn.synsets(word)[0]
    except IndexError:
        best_sense = None
    max_overlap = 0
    context = utils.word_frequencies(utils.preprocess_phrase(sentence))
    for sense in wn.synsets(word):
        debug_trace(f"{pp.info('INFO')}: Considering new sense: {sense}({pp.bold(pp.info(sense.definition()))})")
        examples = sense.examples().copy()
        examples.append(sense.definition())
        debug_trace(f"{pp.warning(pp.bold('EXAMPLES'))}: {examples}")
        signature = utils.word_frequencies(utils.preprocess_phrase(functools.reduce(operator.add, examples)))
        overlap = overlapf(signature, context)
        debug_trace(f"{pp.info('INFO')}: {sense} scored: {pp.success(overlap)}")
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = sense
    return best_sense

Una versione leggermente modificata dell'algoritmo di Lesk, rispetto a quella originale

1. Aggiunge gli 'esempi' di WordNet come parte della signature (più parole ci sono, più dovrebbe essere facile disambiguare)
2. Aumenta le dimensioni del contesto per la signature: prende esempi e definizioni del senso, e dei i suoi iperonimi e iponimi diretti
3. Lascia la parola target nella signature

In [21]:
def lesk_custom_bigcontext(word, sentence, overlapf=computeOverlap):
    try:
        best_sense = wn.synsets(word)[0]
    except IndexError:
        best_sense = None
    max_overlap = 0
    context = utils.word_frequencies(utils.preprocess_phrase(sentence))
    for sense in wn.synsets(word):
        debug_trace(f"{pp.info('INFO')}: Considering new sense: {sense}({pp.bold(pp.info(sense.definition()))})")
        signature = get_big_disambiguation_ctx_for_sysnet(sense)
        overlap = overlapf(signature, context)
        debug_trace(f"{pp.info('INFO')}: {sense} scored: {pp.success(overlap)}")
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = sense
    return best_sense

Una versione leggermente modificata dell'algoritmo di Lesk, rispetto a quella originale

1. Aggiunge gli 'esempi' di WordNet come parte della signature (più parole ci sono, più dovrebbe essere facile disambiguare)
2. Aumenta le dimensioni del contesto per la signature: prende esempi e definizioni del senso, e dei i suoi iperonimi e iponimi diretti
3. Lascia la parola target nella signature
4. Non fa stemming

In [22]:
def lesk_custom_nostem(word, sentence, overlapf=computeOverlap):
    try:
        best_sense = wn.synsets(word)[0]
    except IndexError:
        best_sense = None
    max_overlap = 0
    context = utils.word_frequencies(utils.preprocess_phrase_nostem(sentence))
    for sense in wn.synsets(word):
        if sense.pos() == 'n':
            debug_trace(f"{pp.info('INFO')}: Considering new sense: {sense}({pp.bold(pp.info(sense.definition()))})")
            signature = get_big_disambiguation_ctx_for_sysnet(sense, do_stemming=False)
            overlap = overlapf(signature, context)
            debug_trace(f"{pp.info('INFO')}: {sense} scored: {pp.success(overlap)}")
            if overlap > max_overlap:
                max_overlap = overlap
                best_sense = sense
    return best_sense

Nel blocco seguente, recupero le frasi da SemCor, avendo l'accortezza di recuperare entrambi i tags (semantico e PoS). Si potrebbero contare le frasi (usando il commento) ma ci vuole molto tempo, ho preferito inserirlo a mano.

In [23]:
sentences = sc.tagged_sents(tag='both')
sents_len = 37176 # len(sentences)

La prossima funzione prende una frase da SemCor (rappresentata come un albero), e la restituisce come stringa. (Prende tutte le foglie e le mette in una sola lista, successivamente, 'incolla' i singoli elementi separandoli con uno spazio)

In [24]:
def get_phrase_from_sentence(semcor_sentence):
    """
        Returns a (pseudo-)phrase from a semcor tagged sentence
    """
    return functools.reduce(lambda x, y: x + " " + y, utils.flatten([x.leaves() for x in semcor_sentence]))

In [ ]:
La seguente funzione, prende tutti i possibili alberi da una frase SemCor per recuperarne tutti i sostantivi (usando una depth-first search).

In [25]:
def get_noun_tuples(trees):
    result = []
    """
        Depth-first search that gets (for the trees of a phrase)
    """
    for subtree in trees:
        if type(subtree) == nltk.tree.Tree:
            if type(subtree.label()) == nltk.corpus.reader.wordnet.Lemma:
                if (subtree.label().synset().pos()) == 'n':
                    word = list(subtree.subtrees())[1][0]
                    if type(word) != nltk.tree.Tree:
                        result.append((word, subtree.label()))
            get_noun_tuples(subtree)
    return result

La prossima funzione recupera una lista contenente n numeri randomici. Il parametro `unique` specifica se il risultato può contenere o meno ripetizioni.

In [26]:
def get_n_random_numbers(n, max, unique=True):
    result = []
    if unique:
        for i in range(n):
            r = random.randint(0, max)
            while r in result:
                r = random.randint(0, max)
            result.append(r)
        return result
    else:
        return [random.randint(0, max) for i in range(0, n)]

In [ ]:
La prossima funzione prende una lista di indici e seleziona le frasi di SemCor corrispondenti.

In [ ]:
def random_phrases_from_indexes(indexes):
    result = None
    try:
        result = [sentences[i] for i in indexes]
    except NameError as e:
        sentences = sc.tagged_sents(tag='both')
        result = [sentences[i] for i in indexes]
    return result

### Task \#1: Test drive
 * Estrarre 50 frasi da SemCor
 * Prova a disambiguare almeno un sostantivo per frase
 * Calcola l'accuratezza

In [40]:
def evaluate_accuracy(leskf=lesk):
    indexes = get_n_random_numbers(50, sents_len)
    tagged_sentences = [sentences[i] for i in indexes]
    fifty_sent = [get_phrase_from_sentence(i) for i in tagged_sentences]
    accuracy = 0
    base_accuracy = 0
    for i in range(50):
        targets = get_noun_tuples(tagged_sentences[i])
        if len(targets) > 0:
            nouns, correct = zip(*targets)
            selected_noun = random.randint(0, len(nouns)-1)
            debug = False
            res = leskf(nouns[selected_noun], fifty_sent[i])
            baseres = nltk.wsd.lesk(fifty_sent[i].split(' '), nouns[selected_noun], 'n')
            correct_ss = correct[selected_noun].synset()
            debug = True
            debug_trace(f"Iteration: {i}")
            debug_trace(f"{pp.info('Noun')}: {nouns[selected_noun]} | {pp.info('Phrase')}: {fifty_sent[i]}")
            debug_trace(f"{pp.info('Lesk Result')}: {res}")
            debug_trace(f"{pp.info('Base Lesk Result')}: {baseres}")
            debug_trace(f"{pp.info('Correct result')}: {correct_ss}")
            debug_trace("")
            try:
                accuracy += int(res == correct_ss)/50
                base_accuracy += int(baseres == correct_ss)/50
            except AttributeError as e:
                print(e)
                pass
    print(f"{pp.bold(pp.success('Base Lesk Accuracy'))}: {base_accuracy*100}%")
    print(f"{pp.bold(pp.success('Accuracy'))}: {accuracy*100}%")

In [43]:
debug = False
evaluate_accuracy()

Base Lesk Accuracy: 4.0%
Accuracy: 14.000000000000002%


### Task \#2: Test con randomizzazione
 * Estrarre 50 frasi da SemCor
 * Provare a disambiguare almeno un sostantivo per frase
 * Calcolare l'accuratezza
 * Calcolare l'accuratezza media su _n_ esecuzioni

In [47]:
def round_evaluate(max_runs=50):
    for _ in range(max_runs):
        indexes = get_n_random_numbers(50, sents_len-1)
        tagged_sentences = [sentences[i] for i in indexes]
        fifty_sent = [get_phrase_from_sentence(i) for i in tagged_sentences]

        lesk_baseline_accuracies = []
        lesk_accuracies = []
        lesk_custom_accuracies = []
        lesk_custom_bigcontext_accuracies = []
        lesk_custom_nostem_accuracies = []

        lesk_baseline_accuracy = 0
        lesk_accuracy = 0
        lesk_custom_accuracy = 0
        lesk_custom_bigcontext_accuracy = 0
        lesk_custom_nostem_accuracy = 0
        for i in range(49):
            targets = get_noun_tuples(tagged_sentences[i])
            if len(targets) > 0:
                nouns, correct = zip(*targets)
                selected_noun = random.randint(0, len(nouns)-1)
                res_lesk = lesk(nouns[selected_noun], fifty_sent[i])
                res_lesk_custom = lesk_custom(nouns[selected_noun], fifty_sent[i])
                res_lesk_custom = lesk_custom_bigcontext(nouns[selected_noun], fifty_sent[i])
                res_lesk_custom_nostem = lesk_custom_nostem(nouns[selected_noun], fifty_sent[i])
                correct_ss = correct[selected_noun].synset()
                try:
                    lesk_baseline_accuracy += int(res_lesk == correct_ss)/50
                    lesk_accuracy += int(res_lesk == correct_ss)/50
                    lesk_custom_accuracy += int(res_lesk_custom == correct_ss)/50
                    lesk_custom_bigcontext_accuracy += int(res_lesk_custom == correct_ss)/50
                    lesk_custom_nostem_accuracy += int(res_lesk_custom_nostem == correct_ss)/50
                except AttributeError:
                    pass
        lesk_baseline_accuracies.append(lesk_baseline_accuracy)
        lesk_accuracies.append(lesk_accuracy)
        lesk_custom_accuracies.append(lesk_custom_accuracy)
        lesk_custom_bigcontext_accuracies.append(lesk_custom_bigcontext_accuracy)
        lesk_custom_nostem_accuracies.append(lesk_custom_nostem_accuracy)
    print(f"{pp.bold(pp.success('NLTK Lesk Mean Accuracy'))}: {np.mean(lesk_baseline_accuracies)*100}%")
    print(f"{pp.bold(pp.success('Lesk Mean Accuracy'))}: {np.mean(lesk_accuracies)*100}%")
    print(f"{pp.bold(pp.success('Custom Lesk Mean Accuracy'))}: {np.mean(lesk_custom_accuracies)*100}%")
    print(f"{pp.bold(pp.success('BigContext Custom Lesk Mean Accuracy'))}: {np.mean(lesk_custom_bigcontext_accuracies)*100}%")
    print(f"{pp.bold(pp.success('Nostem Custom Lesk Mean Accuracy'))}: {np.mean(lesk_custom_nostem_accuracies)*100}%")
    return lesk_accuracies, lesk_custom_accuracies, lesk_custom_bigcontext_accuracies, lesk_custom_nostem_accuracies

In [48]:
debug=False
print("Iterations: 50")
round_evaluate(max_runs=50)
print()

Iterations: 50
NLTK Lesk Mean Accuracy: 25.999999999999996%
Lesk Mean Accuracy: 25.999999999999996%
Custom Lesk Mean Accuracy: 36.00000000000001%
BigContext Custom Lesk Mean Accuracy: 36.00000000000001%
Nostem Custom Lesk Mean Accuracy: 40.00000000000001%



In [49]:
print("Iterations: 100")
round_evaluate(max_runs=100)
print()

Iterations: 100
NLTK Lesk Mean Accuracy: 23.999999999999996%
Lesk Mean Accuracy: 23.999999999999996%
Custom Lesk Mean Accuracy: 21.999999999999996%
BigContext Custom Lesk Mean Accuracy: 21.999999999999996%
Nostem Custom Lesk Mean Accuracy: 30.0%



In [50]:
print("Iterations: 1000")
round_evaluate(max_runs=1000)
print()

Iterations: 1000
NLTK Lesk Mean Accuracy: 21.999999999999996%
Lesk Mean Accuracy: 21.999999999999996%
Custom Lesk Mean Accuracy: 25.999999999999996%
BigContext Custom Lesk Mean Accuracy: 25.999999999999996%
Nostem Custom Lesk Mean Accuracy: 34.0%

